## Installing Libraries

In [1]:
## Make sure to change path to the directory where you have cloned IndicTransTokenizer


# %pip install accelerate
# %pip install nltk
# %pip install mosestokenizer
# !pip install evaluate
# !pip install rouge_score
# % pip install  nltk
# !git clone https://github.com/VarunGumma/IndicTransTokenizer
# %cd IndicTransTokenizer
# !pip install --editable /path_to/IndicTransTokenizer

## NLLB Model

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

c:\Users\Tusha\anaconda3\envs\CS689\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Tusha\anaconda3\envs\CS689\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# %%time
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)
# model=model.to(device)
# translator = pipeline('translation', tokenizer=tokenizer ,model=model.to(device), src_lang = 'eng_Latn' ,tgt_lang='hin_Deva' ,max_length=512)
# translation_en_hi = []
# # for x in eng_1000_sentences:
# #     translation_en_hi.append(translator(x)[0]['translation_text'])
# #     print(len(translation_en_hi))
# # inputs = tokenizer(eng_1000_sentences, return_tensors="pt")

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [5]:
# import evaluate 
# def bleu_rouge(ref,pred):
#     bleu = evaluate.load("bleu")
#     rogue = evaluate.load('rouge')
#     #hindi
#     results = bleu.compute(predictions=pred, references=ref)
#     results_r = rogue.compute(predictions=pred, references=ref)
#     return results,results_r

In [6]:
%%time
def translate_sentence(sentence, src_lang, tgt_lang):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt")
    # Generate translation
    translated_ids = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
    # Decode the translated output
    translated_sentence = tokenizer.batch_decode(translated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return translated_sentence[0]




CPU times: total: 0 ns
Wall time: 0 ns


In [2]:
def openRandomDataset(filepath):
    with open(filepath,'r',encoding='utf-8') as f:
        text = f.readlines()
        text = [x.strip().replace('\u200d', '') for x in text]
        
    return text

In [3]:
hindi_1000 = openRandomDataset('../data/hindi_sentences.txt')
eng_1000 = openRandomDataset('../data/eng_sentences.txt')
mar_1000 = openRandomDataset('../data/marathi_sentences.txt')

In [9]:
## Translate English to Hindi
translation_en_hi = []
for x in eng_1000:
    translation_en_hi.append(translate_sentence(x,src_lang='eng_Latn',tgt_lang='hin_Deva'))
print(len(translation_en_hi))

1000


In [11]:
## Translate Hindi to English
translation_hi_en = []
for x in hindi_1000:
    translation_hi_en.append(translate_sentence(x,src_lang='hin_Deva',tgt_lang='eng_Latn'))
print(len(translation_hi_en))

1000


In [12]:
## Translate Hindi to Marathi
translation_hi_mar = []
for x in hindi_1000:
    translation_hi_mar.append(translate_sentence(x,src_lang='hin_Deva',tgt_lang='mar_Deva'))
print(len(translation_hi_mar))

1000


In [13]:
## Translate Marathi to Hindi
translation_mar_hi = []
for x in mar_1000:
    translation_mar_hi.append(translate_sentence(x,src_lang='mar_Deva',tgt_lang='hin_Deva'))
print(len(translation_mar_hi))

1000


## Saving these translated Files

In [14]:
with open("../NLLB/nllb_eng_hindi.txt", "w",encoding='utf-8') as txt_file:
    for line in translation_en_hi:
        txt_file.write(line+'\n')
with open("../NLLB/nllb_hindi_eng.txt", "w",encoding='utf-8') as txt_file:
    for line in translation_hi_en:
        txt_file.write(line+'\n')
with open("../NLLB/nllb_hindi_mar.txt", "w",encoding='utf-8') as txt_file:
    for line in translation_hi_mar:
        txt_file.write(line+'\n')
with open("../NLLB/nllb_mar_hindi.txt", "w",encoding='utf-8') as txt_file:
    for line in translation_mar_hi:
        txt_file.write(line+'\n')

## Computing BLEU score and ROUGE score

In [13]:
## BLEU SCORE ##
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
def bleu_score(ref,pred):
    score = 0
    cc = SmoothingFunction()
    for i in range(len(ref)):
       references = ref[i].split()
       candidate = pred[i].split() 
       score += sentence_bleu([references], candidate,smoothing_function=cc.method2)      
    return score/len(ref)

translation_en_hi = openRandomDataset('../NLLB/nllb_eng_hindi.txt')
translation_hi_en = openRandomDataset('../NLLB/nllb_hindi_eng.txt')
translation_hi_mar = openRandomDataset('../NLLB/nllb_hindi_mar.txt')
translation_mar_hi = openRandomDataset('../NLLB/nllb_mar_hindi.txt')

print("Marathi to Hindi")
print(bleu_score(hindi_1000,translation_mar_hi))
print("Hindi to Marathi")
print(bleu_score(mar_1000,translation_hi_mar))
print("Hindi to English")
print(bleu_score(eng_1000,translation_hi_en))
print("English to Hindi")
print(bleu_score(hindi_1000,translation_en_hi))

Marahti to Hindi
0.2455628308694002
Hindi to Marathi
0.20379466267197663
Hindi to English
0.3531199718526439
English to Hindi
0.3327715489683958


In [15]:
## ROUGE SCORE ##
import evaluate
def rouge_score(ref,pred):
    rouge = evaluate.load('rouge')
    results = rouge.compute(predictions=pred, references=ref)
    return results

print("Marahti to Hindi")
print(rouge_score(hindi_1000,translation_mar_hi))
print("Hindi to Marathi")
print(rouge_score(mar_1000,translation_hi_mar))
print("Hindi to English")
print(rouge_score(eng_1000,translation_hi_en))
print("English to Hindi")
print(rouge_score(hindi_1000,translation_en_hi))


c:\Users\Tusha\anaconda3\envs\CS689\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Marahti to Hindi
{'rouge1': 0.10250000000000001, 'rouge2': 0.02163809523809524, 'rougeL': 0.10212738095238094, 'rougeLsum': 0.10215595238095239}
Hindi to Marathi
{'rouge1': 0.08331904761904763, 'rouge2': 0.019950000000000002, 'rougeL': 0.0834, 'rougeLsum': 0.08247142857142858}
Hindi to English
{'rouge1': 0.6834036052348026, 'rouge2': 0.4537770479890757, 'rougeL': 0.6381618579208067, 'rougeLsum': 0.6384215954682237}
English to Hindi
{'rouge1': 0.11776904761904763, 'rouge2': 0.02590476190476191, 'rougeL': 0.11615833333333336, 'rougeLsum': 0.11665714285714288}
